In [43]:
import pydgraph

client_stub = pydgraph.DgraphClientStub('localhost:9080')
client = pydgraph.DgraphClient(client_stub)

In [20]:
# To set the schema, create an Operation object, set the schema and pass it to DgraphClient#alter(Operation) method.
schema = 'name: string @index(exact) .'
op = pydgraph.Operation(schema=schema)
client.alter(op)

In [21]:
import datetime
import json

import pydgraph


# Create a client stub.
def create_client_stub():
    return pydgraph.DgraphClientStub('localhost:9080')


# Create a client.
def create_client(client_stub):
    return pydgraph.DgraphClient(client_stub)


# Drop All - discard all data and start from a clean slate.
def drop_all(client):
    return client.alter(pydgraph.Operation(drop_all=True))


# Set schema.
def set_schema(client):
    schema = """
    name: string @index(exact) .
    friend: uid @reverse .
    age: int .
    married: bool .
    loc: geo .
    dob: datetime .
    """
    return client.alter(pydgraph.Operation(schema=schema))


# Create data using JSON.
def create_data(client):
    # Create a new transaction.
    txn = client.txn()
    try:
        # Create data.
        p = {
            'name': 'Alice',
            'age': 26,
            'married': True,
            'loc': {
                'type': 'Point',
                'coordinates': [1.1, 2],
            },
            'dob': datetime.datetime(1980, 1, 1, 23, 0, 0, 0).isoformat(),
            'friend': [
                {
                    'name': 'Bob',
                    'age': 24,
                },
                {
                    'name': 'Charlie',
                    'age': 29,
                }
            ],
            'school': [
                {
                    'name': 'Crown Public School',
                }
            ]
        }

        # Run mutation.
        assigned = txn.mutate(set_obj=p)

        # Commit transaction.
        txn.commit()

        # Get uid of the outermost object (person named "Alice").
        # assigned.uids returns a map from blank node names to uids.
        # For a json mutation, blank node names "blank-0", "blank-1", ... are used
        # for all the created nodes.
        print('Created person named "Alice" with uid = {}\n'.format(assigned.uids['blank-0']))

        print('All created nodes (map from blank node names to uids):')
        for uid in assigned.uids:
            print('{} => {}'.format(uid, assigned.uids[uid]))
    finally:
        # Clean up. Calling this after txn.commit() is a no-op
        # and hence safe.
        txn.discard()
        print('\n')


#Deleting a data
def delete_data(client):
    # Create a new transaction.
    txn = client.txn()
    try:
        query1 = """query all($a: string)
        {
           all(func: eq(name, $a)) 
            {
               uid
            }   
        }"""
        variables1 = {'$a': 'Bob'}
        res1 = client.txn(read_only=True).query(query1, variables=variables1)
        ppl1 = json.loads(res1.json)
        for person in ppl1['all']:
            print('Query to find Uid for Bob :')
            print(query1)
            print('\n')
            print("Bob's UID : ")
            print(person)
            print('\n')
            print('Bob deleted')
            print('\n')


        assigned = txn.mutate(del_obj= person)

        txn.commit()

    finally:
        txn.discard()


# Query for data.
def query_data(client):
    # Run query.
    query = """query all($a: string) {
        all(func: eq(name, $a)) {
            uid
            name
            age
            married
            loc
            dob
            friend {
                name
                age
            }
            school {
                name
            }
        }
    }"""

    variables = {'$a': 'Alice'}
    res = client.txn(read_only=True).query(query, variables=variables)
    ppl = json.loads(res.json)

    # Print results.
    print('Number of people named "Alice": {}'.format(len(ppl['all'])))
    print('\n')
    for person in ppl['all']:
        print('Query for Alice : \n' +query)
        print('\n')
        print('Result :')
        print(person)
        print('\n')

#Query to check for deleted node
def query_data01(client):
    query01 = """query all($b: string)
        {   all(func: eq(name, $b)) 
            {   uid, 
                name,
                age
                friend 
                { 
                    uid,
                    name,
                    age 
                }
                ~friend 
                { 
                    uid,
                    name,
                    age 
                }
            }   
        }"""

    variables01 = {'$b': 'Bob'}
    res01 = client.txn(read_only=True).query(query01, variables=variables01)
    ppl01 = json.loads(res01.json)

    print('Number of people named "Bob": {}'.format(len(ppl01['all'])))
    print('\n')
    for person in ppl01['all']:
        print('Query for Bob :\n' + query01)
        print('\n')
        print('Result :')
        print(person)
        print('\n')

In [22]:
client_stub = create_client_stub()
client = create_client(client_stub)
drop_all(client)
set_schema(client)
create_data(client)

Created person named "Alice" with uid = 0x2730

All created nodes (map from blank node names to uids):
blank-1 => 0x2731
blank-2 => 0x2732
blank-3 => 0x2733
blank-0 => 0x2730




In [23]:
query_data(client) # query for Alice

Number of people named "Alice": 1


Query for Alice : 
query all($a: string) {
        all(func: eq(name, $a)) {
            uid
            name
            age
            married
            loc
            dob
            friend {
                name
                age
            }
            school {
                name
            }
        }
    }


Result :
{'uid': '0x2730', 'name': 'Alice', 'age': 26, 'married': True, 'loc': {'type': 'Point', 'coordinates': [1.1, 2]}, 'dob': '1980-01-01T23:00:00Z', 'friend': [{'name': 'Bob', 'age': 24}, {'name': 'Charlie', 'age': 29}], 'school': [{'name': 'Crown Public School'}]}




In [24]:
query_data01(client) # query for Bob

Number of people named "Bob": 1


Query for Bob :
query all($b: string)
        {   all(func: eq(name, $b)) 
            {   uid, 
                name,
                age
                friend 
                { 
                    uid,
                    name,
                    age 
                }
                ~friend 
                { 
                    uid,
                    name,
                    age 
                }
            }   
        }


Result :
{'uid': '0x2731', 'name': 'Bob', 'age': 24, '~friend': [{'uid': '0x2730', 'name': 'Alice', 'age': 26}]}




In [25]:
delete_data(client) # delete Bob

Query to find Uid for Bob :
query all($a: string)
        {
           all(func: eq(name, $a)) 
            {
               uid
            }   
        }


Bob's UID : 
{'uid': '0x2731'}


Bob deleted




In [26]:
query_data(client) # query for Alice

Number of people named "Alice": 1


Query for Alice : 
query all($a: string) {
        all(func: eq(name, $a)) {
            uid
            name
            age
            married
            loc
            dob
            friend {
                name
                age
            }
            school {
                name
            }
        }
    }


Result :
{'uid': '0x2730', 'name': 'Alice', 'age': 26, 'married': True, 'loc': {'type': 'Point', 'coordinates': [1.1, 2]}, 'dob': '1980-01-01T23:00:00Z', 'friend': [{'name': 'Charlie', 'age': 29}], 'school': [{'name': 'Crown Public School'}]}




In [27]:
query_data01(client) # query for Bob

Number of people named "Bob": 0




In [31]:
query = """{
  find_person(func: eq(name, "Alice")) {
    uid
    name
    age
  }
}
"""
# variables = {'$a': 'Alice'}
res = client.txn(read_only=True).query(query)
ppl = json.loads(res.json)
print(json.dumps(ppl, indent=True))

{
 "find_person": [
  {
   "uid": "0x2730",
   "name": "Alice",
   "age": 26
  }
 ]
}


In [40]:
def create_data(client):
    # Create a new transaction.
    txn = client.txn()
    try:
        # Create data.
        p = {
            'uid': '_:UHJvZHVjdFR5cGU6+TUFS/S0VUSU5HX1BLRw==',
            'name': 'Alice',
            'age': 26,
            'married': True,
            'loc': {
                'type': 'Point',
                'coordinates': [1.1, 2],
            },
            'dob': datetime.datetime(1980, 1, 1, 23, 0, 0, 0).isoformat(),
            'friend': [
                {
                    'name': 'Bob',
                    'age': 24,
                },
                {
                    'name': 'Charlie',
                    'age': 29,
                }
            ],
            'school': [
                {
                    'name': 'Crown Public School',
                }
            ]
        }

        # Run mutation.
        assigned = txn.mutate(set_obj=p)

        # Commit transaction.
        txn.commit()

        # Get uid of the outermost object (person named "Alice").
        # assigned.uids returns a map from blank node names to uids.
        # For a json mutation, blank node names "blank-0", "blank-1", ... are used
        # for all the created nodes.
        print('Created person named "Alice" with uid = {}\n'.format(assigned.uids['blank-0']))

        print('All created nodes (map from blank node names to uids):')
        for uid in assigned.uids:
            print('{} => {}'.format(uid, assigned.uids[uid]))
    finally:
        # Clean up. Calling this after txn.commit() is a no-op
        # and hence safe.
        txn.discard()
        print('\n')

In [41]:
client_stub = create_client_stub()
client = create_client(client_stub)
drop_all(client)
set_schema(client)
create_data(client)

Created person named "Alice" with uid = 0x2745

All created nodes (map from blank node names to uids):
blank-1 => 0x2746
blank-2 => 0x2747
UHJvZHVjdFR5cGU6+TUFS/S0VUSU5HX1BLRw== => 0x2744
blank-0 => 0x2745


